In [14]:
import torch

In [25]:
x = torch.rand(32, 128)
y = torch.rand(32, 128) / 10 + x

In [26]:
cos = torch.nn.CosineSimilarity()

In [27]:
cos(x, y)

tensor([0.9977, 0.9983, 0.9980, 0.9984, 0.9983, 0.9982, 0.9975, 0.9982, 0.9985,
        0.9983, 0.9978, 0.9979, 0.9984, 0.9982, 0.9983, 0.9984, 0.9981, 0.9980,
        0.9982, 0.9983, 0.9985, 0.9979, 0.9982, 0.9983, 0.9980, 0.9981, 0.9976,
        0.9983, 0.9983, 0.9979, 0.9981, 0.9977])

In [28]:
kl = torch.nn.KLDivLoss(reduction='batchmean')

In [30]:
kl(x, torch.rand(32, 128))

tensor(-63.7799)

In [18]:
x.mean()

tensor(0.4917)

In [19]:
x.std()

tensor(0.2879)

In [21]:
(torch.rand(32, 128) / 100).std()

tensor(0.0030)

In [1]:
import numpy as np
import faiss
import json
from tqdm import tqdm
from augmentation import TfIdfAugmentation
from tools.utils import ExternalPreprocessor

In [2]:
VOCAB_PATH = '../routing/data/sberbank_embeddings/w2v_m5_w3_v300_norm_v48_vocab.txt'
W2V_MATRIX_PATH = '../routing/data/sberbank_embeddings/w2v_m5_w3_v300_norm_v48_vectors.npy'

In [3]:
with open('token2prob.json') as f:
    token2prob = json.loads(f.read())

In [4]:
augmentator = TfIdfAugmentation(vocab_path=VOCAB_PATH, matrix_path=W2V_MATRIX_PATH, token2prob=token2prob)

In [5]:
with open('../routing/data/dataset/data.jsonl') as f:
    train = [json.loads(sample) for sample in f.read().split('\n')]

In [6]:
processor = ExternalPreprocessor()

In [7]:
origin = [processor.return_lemmas_only(sample['parsed']['tokenized_elements_list']) for sample in tqdm(train)]

100%|██████████| 12915/12915 [00:00<00:00, 78243.92it/s]


In [11]:
train_json = [{
    'text': train[n]['text'],
    'target': train[n]['target'],
    'tokens': origin[n],
} for n in range(len(train))]

In [13]:
with open('train.jsonl', 'w') as f:
    f.write('\n'.join([json.dumps(sample, ensure_ascii=False) for sample in train_json]))

In [8]:
augmented = [augmentator(sample) for sample in tqdm(origin)]

  2%|▏         | 196/12915 [00:13<17:23, 12.19it/s]

KeyboardInterrupt: 

  2%|▏         | 196/12915 [00:30<17:23, 12.19it/s]

In [ ]:
similarity = [sum([1 if origin[n][i] == augmented[n][i] else 0 for i in range(len(origin[n]))]) / len(origin[n])
              for n in range(len(origin))]

In [ ]:
similarity[:5]

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(similarity)

In [13]:
similarity.count(1.) * 100 / len(similarity)

0.01548586914440573

In [14]:
full_sim_indexes = [w for w in range(len(similarity)) if similarity[w] == 1.]

In [15]:
origin[full_sim_indexes[0]]

['проверять', 'проконсультироваться', 'кешкарта', '.']

In [16]:
origin[full_sim_indexes[0]]

['проверять', 'проконсультироваться', 'кешкарта', '.']

In [17]:
import pandas as pd

In [24]:
n = 6
print(sum([1 if origin[n][i] == augmented[n][i] else 0 for i in range(len(origin[n]))]) / len(origin[n]))

pd.DataFrame(data={'origin': origin[n], 'augmented': augmented[n]})

0.2857142857142857


,origin,augmented
0,интересовать,интересовать
1,открытие,отрытие
2,расчетный_счет,карточный_счет
3,как,как-то
4,индивидуальный,общеустановленный
5,предприниматель,бизнесмен
6,.,.


In [23]:
augmentator.k_top_words

5

In [24]:
augmentator.replace_word('кредит')

'заем'

In [25]:
ind = augmentator.get_word_index('кредит')

In [27]:
word_vector = augmentator.get_word_vector('кредит')

In [33]:
c, v = augmentator.index.search(np.array([word_vector]), 5)

In [34]:
v

array([[  34,  606,  161, 1020, 2166]])

In [36]:
augmentator.get_similar('кредит', 5)

['потребительский_кредит', 'ипотека', 'заем', 'автокредит', 'потребкредит']

In [38]:
origin[2]

['вы',
 'просить',
 'кредит',
 'кредит',
 'как',
 'практически',
 'одобрять',
 'ну',
 'как-то',
 'просто',
 'вопрос',
 'задавать',
 'большой',
 'цифра',
 'вы',
 'требовать',
 'поручитель',
 'поручитель',
 'юрлицо',
 'обязательно',
 'поручитель',
 'приезжать',
 'к',
 'вы',
 'когда',
 'брать',
 'кредит',
 'договор',
 'составлять',
 'или',
 'как-то',
 'просто',
 'давать',
 'информация',
 'немой',
 '.']

In [63]:
augmentator.replace_sentence(origin[2])

['он',
 'просить',
 'потребительский_кредит',
 'потребительский_кредит',
 'как-то',
 'почти',
 'одобрять',
 'да',
 'зимпальто',
 'тупо',
 'ворос',
 'задавать',
 'маленький',
 'цифирь',
 'он',
 'требовать',
 'созаемщик',
 'созаемщик',
 'юр_лицо',
 'непременно',
 'созаемщик',
 'приезжать',
 'человек-заемщик',
 'он',
 'TIME_TEMPORAL_TOKEN',
 'брать',
 'потребительский_кредит',
 'кредитный_договор',
 'составлять',
 'либо',
 'зимпальто',
 'тупо',
 'давать',
 'сведение',
 'бальзамовый',
 '.']

In [41]:
import random

In [50]:
[random.choice(index) for index in v[:, 1:]]

[1020]